# Pull in player data from Soccerbase

In [1]:
import pandas as pd

sb_df = pd.read_csv('https://raw.githubusercontent.com/petebrown/scrape-player-info/main/data/player-info.csv')

sb_plr_ids = sb_df.player_id.to_list()

In [2]:
import concurrent.futures
from bs4 import BeautifulSoup
import requests
from tqdm.notebook import tqdm


headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

def get_soup(url):
    response = requests.get(url, headers=headers)
    return BeautifulSoup(response.text, "html.parser")

def get_info(url):
    pl = pd.concat([pd.read_html(url)[1], pd.read_html(url)[2]]).dropna()
    pl.columns = ['stat', 'value']
    pl['player_id'] = url.split('=')[-1]
    pl = pl.pivot(index='player_id', columns='stat', values='value').reset_index()
    return pl

def fetch_player_data(id):
    url = f"https://www.soccerbase.com/players/player.sd?player_id={id}"
    soup = get_soup(url)
    name = soup.find('table', {"class": 'imageHead'}).find('h1').text
    try:
        position = soup.find('div', class_='midfielder bull').get_text(strip=True, separator=' ').split(' ')[0]
    except:
        position = ''
    return {
        'player_id': id,
        'player_name': name.strip(),
        'soccerbase_pos': position,
        'player_info': get_info(url)
    }

player_info = []

with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = {executor.submit(fetch_player_data, id): id for id in sb_plr_ids}
    for future in tqdm(concurrent.futures.as_completed(futures)):
        player_data = future.result()
        player_info.append(player_data)

0it [00:00, ?it/s]

In [22]:
df = pd.DataFrame(player_info)

df.player_id = df.player_id.astype(int)

# Remove shirt numbers from scraped player names
df['player_name'] = df['player_name'].str.replace('^\\d+. ', '', regex=True)

In [23]:
name_fixes = {
    "Anthony Kay": "Antony Kay",
    "Corey Taylor": "Corey Blackett-Taylor",
    "Craig Carl Curran": "Craig Curran",
    "Chris Edwards": "Christian Edwards",
    "Daniel Robert Harrison": "Danny Harrison",
    "Dave Nugent": "David Nugent",
    "Dylan Mottley Henry": "Dylan Mottley-Henry",
    "Jack Flemming": "Jack Fleming",
    "Jay Devine": "James Devine",
    "Jay Turner-Cook": "Jay Turner-Cooke",
    "Jimmy McNulty": "Jim McNulty",
    "John-Louis Akpa Akpro": "Jean-Louis Akpa Akpro",
    "John Morrissey": "Johnny Morrissey",
    "Jonathon Margetts": "Johnny Margetts",
    "Joseph Maguire": "Joe Maguire",
    "Kaylden Brown": "Kayleden Brown",
    "Lewis Sinnot": "Lewis Sinnott",
    "Lateef Elford Alliyu": "Lateef Elford-Alliyu",
    "Matty Kennedy": "Matthew Kennedy",
    "Michael Jackson": "Mike Jackson",
    "Michael Jones": "Mike Jones",
    "Oliver Banks": "Ollie Banks",
    "Ousmane Kane": "Ousmane Kané",
    "Richard Sutton": "Ritchie Sutton",
    "Robert Apter": "Rob Apter",
    "Robert Taylor": "Rob Taylor",
    "Steven O'Leary": "Stephen O'Leary",
    "Tom Coughan": "Tom Croughan"
}

for old_name, new_name in tqdm(name_fixes.items()):
    df.loc[df.player_name == old_name, 'player_name'] = new_name

  0%|          | 0/28 [00:00<?, ?it/s]

In [24]:
sb_plr_names = df[['player_id', 'player_name']].copy()

sb_plr_names['player_id'] = sb_plr_names['player_id'].astype(int)

sb_plr_names.head(5)

,player_id,player_name
0,197047,Reuben Egan
1,102629,Declan Drysdale
2,123720,Saidou Khan
3,81221,Sam Finley
4,101331,Josh Hawkes


In [25]:
sb_pos_fixes = {
    "Billy Woods": "Midfielder",
    "Dariusz Kubicki": "Defender",
    "Dave Higgins": "Defender",
    "Gary Stevens": "Defender",
    "Andy Thorn": "Defender",
    "James Devine": "Midfielder",
    "Stephen O'Leary": "Midfielder",
    "Steve Mungall": "Defender"
}

sb_plr_pos = df[['player_id', 'player_name', 'soccerbase_pos']].copy()

for plr, pos in sb_pos_fixes.items():
    sb_plr_pos.loc[sb_plr_pos.player_name == plr, 'soccerbase_pos'] = pos

sb_plr_pos.head(3)

,player_id,player_name,soccerbase_pos
0,197047,Reuben Egan,Goalkeeper
1,102629,Declan Drysdale,Defender
2,123720,Saidou Khan,Midfielder


In [26]:
plr_info = pd.concat(df['player_info'].tolist(), ignore_index=True)

plr_info['dob'] = plr_info.Age.str.extract(r'\d+ \(Born (.*)\)')
plr_info['dob'] = pd.to_datetime(plr_info['dob'])

sb_plr_dobs = plr_info[['player_id', 'dob']].copy()

sb_plr_dobs.player_id = sb_plr_dobs.player_id.astype(int)

sb_plr_dobs.head(5)

stat,player_id,dob
0,197047,2005-07-27
1,102629,1999-11-14
2,123720,1995-12-05
3,81221,1992-08-04
4,101331,1999-01-28


In [27]:
sb_plrs = sb_plr_names.merge(sb_plr_dobs, on='player_id')

sb_plrs

,player_id,player_name,dob
0,197047,Reuben Egan,2005-07-27
1,102629,Declan Drysdale,1999-11-14
2,123720,Saidou Khan,1995-12-05
3,81221,Sam Finley,1992-08-04
4,101331,Josh Hawkes,1999-01-28
...,...,...,...
483,925,Ged Brannan,1972-01-15
484,24422,Seyni N'Diaye,NaT
485,7838,Shaun Teale,1964-03-10
486,9704,Paul Shepherd,1977-11-17


In [28]:
dob_fixes = {
    'Akpo Sodje': '1980-01-31',
    'Arnaud Mendy': '1990-02-10',
    'Ben Tomlinson': '1989-10-31',
    'Brad Walker': '1996-04-25',
    'Callum Lucy': '1998-11-09',
    'Chris McCready': '1981-09-05',
    'Clayton McDonald': '1988-12-06',
    'Ernie Davies': '1916-01-31',
    'Emmanuel Dieseruvwe': '1995-02-20',
    'Jack Fleming': '1999-01-10',
    'Kane Hemmings': '1991-04-08',
    'Mikey Davies': '2004-09-23',
    'Nathan Blissett': '1990-06-29',
    'Robbie Burns': '1990-11-15',
    'Perry Taylor': '1981-01-29',    
    'Rob Apter': '2003-04-23',
    'Ryan Edwards': '1993-10-07',
    'Sam Ilesanmi': '1998-07-13',
    'Scott Davies': '1987-02-23',
    "Stephen O'Leary": '1987-02-02',
    "Seyni N'Diaye": '1973-06-01',
    'Tom Croughan': '1999-09-19',
    'Will Aimson': '1994-06-03'
}

for name, dob in dob_fixes.items():
    sb_plrs.loc[sb_plrs.player_name == name, 'dob'] = dob

sb_plrs

,player_id,player_name,dob
0,197047,Reuben Egan,2005-07-27
1,102629,Declan Drysdale,1999-11-14
2,123720,Saidou Khan,1995-12-05
3,81221,Sam Finley,1992-08-04
4,101331,Josh Hawkes,1999-01-28
...,...,...,...
483,925,Ged Brannan,1972-01-15
484,24422,Seyni N'Diaye,1973-06-01
485,7838,Shaun Teale,1964-03-10
486,9704,Paul Shepherd,1977-11-17


In [29]:
sb_plrs[~sb_plrs.dob.notnull()].reset_index(drop=True)

,player_id,player_name,dob
0,146009,Dylan Dwyer,NaT
1,134764,Jamie Timlin,NaT
2,111843,Nick Long,NaT
3,111841,James Devine,NaT
4,102625,James Divine,NaT
5,88395,Darren Askew,NaT
6,86848,Andy Mathieson,NaT
7,61489,John Courtney,NaT
8,51901,Thomas Baker,NaT
9,51109,Peter Kennedy,NaT


In [30]:
sb_plrs = sb_plrs[sb_plrs.dob.notnull()].reset_index(drop=True)

sb_plrs

,player_id,player_name,dob
0,197047,Reuben Egan,2005-07-27
1,102629,Declan Drysdale,1999-11-14
2,123720,Saidou Khan,1995-12-05
3,81221,Sam Finley,1992-08-04
4,101331,Josh Hawkes,1999-01-28
...,...,...,...
469,925,Ged Brannan,1972-01-15
470,24422,Seyni N'Diaye,1973-06-01
471,7838,Shaun Teale,1964-03-10
472,9704,Paul Shepherd,1977-11-17


In [31]:
def split_name(name):
    if name == 'Pedro Miguel Matias':
        forename = 'Pedro Miguel'
        surname = 'Matias'
    else:
        parts = name.split()
        forename = parts[0]
        surname = ' '.join(parts[1:])
    return pd.Series([forename, surname])
    
sb_plrs[['forename', 'surname']] = sb_plrs['player_name'].apply(split_name)

sb_plrs.head(3)

,player_id,player_name,dob,forename,surname
0,197047,Reuben Egan,2005-07-27,Reuben,Egan
1,102629,Declan Drysdale,1999-11-14,Declan,Drysdale
2,123720,Saidou Khan,1995-12-05,Saidou,Khan


In [32]:
sb_plrs = sb_plrs.merge(sb_plr_pos, on=['player_id', 'player_name'], how='left')

In [33]:
sb_plrs['player_id'] = sb_plrs.surname + sb_plrs.forename + sb_plrs.dob.astype(str)

sb_plrs['player_id'] = sb_plrs['player_id'].str.replace('.0', '').str.replace('-', '').str.replace("'", '').str.replace(" ", '').str.replace('é', 'e')

sb_plrs = sb_plrs[['player_id', 'surname', 'forename', 'player_name', 'dob', 'soccerbase_pos']].copy()

sb_plrs

,player_id,surname,forename,player_name,dob,soccerbase_pos
0,EganReuben20050727,Egan,Reuben,Reuben Egan,2005-07-27,Goalkeeper
1,DrysdaleDeclan19991114,Drysdale,Declan,Declan Drysdale,1999-11-14,Defender
2,KhanSaidou19951205,Khan,Saidou,Saidou Khan,1995-12-05,Midfielder
3,FinleySam19920804,Finley,Sam,Sam Finley,1992-08-04,Midfielder
4,HawkesJosh19990128,Hawkes,Josh,Josh Hawkes,1999-01-28,Midfielder
...,...,...,...,...,...,...
469,BrannanGed19720115,Brannan,Ged,Ged Brannan,1972-01-15,Midfielder
470,NDiayeSeyni19730601,N'Diaye,Seyni,Seyni N'Diaye,1973-06-01,Forward
471,TealeShaun19640310,Teale,Shaun,Shaun Teale,1964-03-10,Defender
472,ShepherdPaul19771117,Shepherd,Paul,Paul Shepherd,1977-11-17,Forward


In [34]:
sb_pos_codes = {
    'Goalkeeper': 'G',
    'Defender': 'D',
    'Midfielder': 'MF',
    'Forward': 'F'
}

sb_plrs['position'] = sb_plrs.soccerbase_pos.map(sb_pos_codes)

In [35]:
sb_plrs = sb_plrs.rename(columns={'dob': 'player_dob'})

In [36]:
sb_plrs = sb_plrs[['player_id', 'surname', 'forename', 'player_name', 'player_dob', 'soccerbase_pos']].sort_values('player_id').reset_index(drop=True)

sb_plrs.head(5)

,player_id,surname,forename,player_name,player_dob,soccerbase_pos
0,AchterbergJohn19710708,Achterberg,John,John Achterberg,1971-07-08,Goalkeeper
1,AhmedAdnan19840607,Ahmed,Adnan,Adnan Ahmed,1984-06-07,Midfielder
2,AimsonWill19940603,Aimson,Will,Will Aimson,1994-06-03,Defender
3,AistonSam19761121,Aiston,Sam,Sam Aiston,1976-11-21,Midfielder
4,AkammaduFranklyn19980811,Akammadu,Franklyn,Franklyn Akammadu,1998-08-11,Forward


In [37]:
sb_plrs.to_csv('./output/soccerbase.csv', index=False)

In [19]:
# r_ids = pd.read_csv('~/Desktop/player_ids.csv')[['player_id', 'player_name']].drop_duplicates().reset_index(drop=True)

# r_ids

In [20]:
# r_ids.query("~player_id.isin(@sb_plrs.player_id)")

In [21]:
# sb_plrs.query("~player_id.isin(@r_ids.player_id)").sort_values(['surname', 'forename'])